In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
doctors = pd.read_csv('doctors.csv'); 
example = pd.read_csv('example.csv');
example

,Image Index,Finding Label,Bbox [x,y,w,h]
0,1.png,Cardiomegaly,363.932203,470.002268,451.254237,271.186441
1,2.png,Cardiomegaly,251.394709,435.606349,497.371429,430.188360
2,3.png,Effusion,149.617778,728.500156,27.306667,65.991111


In [3]:
doctors

,id,original,on_created,doctor1_id,doctor1_mark,on_doctor1_created,on_doctor1_updated,doctor2_id,doctor2_mark,on_doctor2_created,on_doctor2_updated,doctor3_id,doctor3_mark,on_doctor3_created,on_doctor3_updated
0,5155,"{""url"":""/data/mask/1.png"",""name"":""1.png""}",8/26/2019 12:00:33 PM,NaN,NaN,NaN,NaN,8.0,"{""marks"":[{""symptom"":""cardiomegaly"",""x"":38.877...",8/26/2019 6:39:14 PM,8/26/2019 6:39:14 PM,NaN,NaN,NaN,NaN
1,5217,"{""url"":""/data/mask/28.png"",""name"":""2.png""}",8/26/2019 12:00:33 PM,7.0,"{""marks"":[{""symptom"":""cardiomegaly"",""x"":26.352...",8/27/2019 6:15:24 PM,8/27/2019 6:15:24 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5299,"{""url"":""/data/mask/3.png"",""name"":""3.png""}",8/26/2019 12:00:33 PM,NaN,NaN,NaN,NaN,8.0,"{""marks"":[{""symptom"":""pneumonia"",""x"":58.486706...",8/30/2019 9:22:05 PM,8/30/2019 9:22:05 PM,NaN,NaN,NaN,NaN


In [4]:
def bb_intersection_over_union(boxA, boxB, boxA_weight, boxA_height, boxB_weight, boxB_height):
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])

    print(x1,y1,x2,y2)
    interArea = max((x2 - x1), 0) * max((y2 - y1), 0)
    
    boxAArea = abs(boxA_weight*boxA_height)
    boxBArea = abs(boxB_weight*boxB_height)
    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    return iou

In [5]:
bb_intersection_over_union([149.72596224, 716.171776-58.633943040000005, 149.72596224+24.081799168, 716.171776], [149.617777777778, 728.50015625-65.9911111111111, 149.617777777778+27.306666666666697, 728.50015625], 24.081799168, 58.633943040000005, 27.306666666666697, 65.9911111111111)*100 

149.72596224 662.509045138889 173.807761408 716.171776


67.24703961703827

In [6]:
for doctor in ['doctor1', 'doctor2', 'doctor3']:
    for i, mark in enumerate(doctors[doctor+'_mark']):
        if not pd.isnull(mark):
            marks = json.loads(mark)['marks']
            for symptom in marks:
                example_cor = example.iloc[i]
                print("Diagnosed by ", doctor, " for image", str(i)+',', "Symptom:", symptom['symptom'])
                print(symptom['x']*10.24, symptom['y']*10.24, symptom['width']*10.24, symptom['height']*10.24)
                print(example_cor['Bbox [x'], example_cor['y'], example_cor['w'], example_cor['h]'])
             
                predicted_box = [symptom['x']*10.24, symptom['y']*10.24-symptom['height']*10.24, symptom['x']*10.24+symptom['width']*10.24, symptom['y']*10.24]
                actual_box = [example_cor['Bbox [x'], example_cor['y']-example_cor['h]'], example_cor['Bbox [x']+example_cor['w'], example_cor['y']]
                
                iou = bb_intersection_over_union(predicted_box, actual_box, symptom['width']*10.24, symptom['height']*10.24, example_cor['w'], example_cor['h]'])
                print(iou*100, '\n')
        

Diagnosed by  doctor1  for image 1, Symptom: cardiomegaly
269.8516992 367.32663808 445.30660352 382.71744
251.39470899470902 435.606349206349 497.37142857142896 430.18835978836
269.8516992 5.41798941798902 715.1583027199999 367.32663808
72.19483672303589 

Diagnosed by  doctor2  for image 0, Symptom: cardiomegaly
398.10820096000003 563.3026048 392.9779712 165.1943936
363.932203389831 470.002267611229 451.254237288136 271.186440677966
398.10820096000003 398.1082112 791.08617216 470.002267611229
17.764686062514706 

Diagnosed by  doctor2  for image 0, Symptom: consolidation
591.0060032 390.92585472 91.3186304 129.28257024
363.932203389831 470.002267611229 451.254237288136 271.186440677966
591.0060032 261.64328448000003 682.3246336 390.92585472
9.647395942911187 

Diagnosed by  doctor2  for image 0, Symptom: consolidation
342.70140416000004 438.12425728000005 59.51101952 144.67335168
363.932203389831 470.002267611229 451.254237288136 271.186440677966
363.932203389831 293.45090560000006 40